In [1]:
# my python auto generate code for some things
import numpy as np
import glob
from PIL import Image
import os
import random
import fnmatch
import glob
import re

work_dir = r'output\dir\path'

In [ ]:
# auto upgrade links for autobuild
target_dir = r'target\dir\path'

def make_upgrade(files):
    for file in files:
        with open(file, 'r') as f:
            text = f.read()
        text = re.sub('#+.*\n', '\n', text)
        text = re.sub('\t*\n', '\n', text)
        text = re.sub(' *\n', '\n', text)
        buildings = re.findall(r'\n\w*? = {.*?\n}', text, re.DOTALL)
        for building in buildings:
            if 'upgrades = {' in building:
                name = re.match(r'\n\w* = {', building).group(0).replace(' = {', '')
                print(name)
                upgrade = re.search(r'upgrades = {.*?}', building, re.DOTALL).group(0).replace('upgrades = {', '').replace('}', '').replace('\t', '')
                upgrade = upgrade.split()
                print(upgrade)
                if len(upgrade) == 1:
                    with open(os.path.join(work_dir, 'update_code'), 'a') as f2:
                        f2.write('{0} = {{ add_building_construction = {1} }}'.format(name, upgrade[0]))

make_upgrade(glob.glob(target_dir + '\\*.txt'))
print('done')

In [ ]:
# clean blocker code generation for autobuild
target_dir = r'target\dir\path'
ban_list = []
def make_blocker(files):
    for file in files:
        with open(file, 'r') as f:
            text = f.readlines()
        i = 0
        name = None
        cost = 0
        tech = None
        while i < len(text):
            line_temp = text[i][:text[i].find('#')].replace('\n', '')
            if len(line_temp) > 0:
                if not line_temp.startswith('\t') and not line_temp.startswith('}'):
                    name = line_temp.replace(' = {', '')
                if line_temp.startswith('\t\tenergy = '):
                    cost = line_temp.replace('\t', '').replace('energy = ', '')
                if line_temp.startswith('\tprerequisites'):
                    tech = line_temp.replace('\t', '\n' + '\t'*8).replace('prerequisites', 'has_technology').replace('{ "', '').replace('" }', '')
                if line_temp.startswith('}'):
                    try:
                        if name != None and int(cost) > 0:
                            #print(name)
                            with open(os.path.join(work_dir, 'blocker_code'), 'a') as f2:
                                f2.write('\t\t\t\t\tif = {{\n\t\t\t\t\t\tlimit = {{\n\t\t\t\t\t\t\thas_blocker = {0}\n\t\t\t\t\t\t\towner = {{\n\t\t\t\t\t\t\t\tenergy > {2}{1}\n\t\t\t\t\t\t\t}}\n\t\t\t\t\t\t}}\n\t\t\t\t\t\towner = {{\n\t\t\t\t\t\t\tif = {{\n\t\t\t\t\t\t\t\tlimit = {{\n\t\t\t\t\t\t\t\t\thas_ascension_perk = ap_mastery_of_nature\n\t\t\t\t\t\t\t\t}}\n\t\t\t\t\t\t\t\tadd_energy = -{3}\n\t\t\t\t\t\t\t\telse = {{\n\t\t\t\t\t\t\t\t\tadd_energy = {2} \n\t\t\t\t\t\t\t\t}}\n\t\t\t\t\t\t\t}}\n\t\t\t\t\t\t}}\n\t\t\t\t\t\tremove_blocker = yes\n\t\t\t\t\t}}\n'.format(name, str(tech).replace('None', ''), cost, str(int(int(cost)*0.67))))
                        name = None
                        cost = 0
                        tech = None
                    except Exception as e:
                        print(e)
            i += 1

make_blocker(glob.glob(target_dir + '\\*.txt'))
print('done')

In [ ]:
# generate code for add leader trait on lvl up, usefull for mods with new traits
target_dir = r'target\dir\path'
def make_trait(file):
    with open(file, 'r') as f:
        text = f.readlines()
    i = 0
    name = None
    random = True
    leader_class = []
    trigger = ''
    while i < len(text):
        line_temp = text[i][:text[i].find('#')].replace('\n', '')
        if len(line_temp) > 0:
            if not line_temp.startswith('\t') and not line_temp.startswith('}'):
                name = line_temp.replace(' = {', '')
            if line_temp.startswith('\trandomized = no'):
                random = False
            if line_temp.startswith('\tleader_potential_add = {'):
                while True:
                    i += 1
                    line_temp = text[i][:text[i].find('#')]
                    if line_temp.startswith('\t}'):
                        break
                    else:
                        trigger += '\n' + line_temp
            if line_temp.startswith('\tleader_class = {'):
                while True:
                    i += 1
                    line_temp = text[i][:text[i].find('#')]
                    if line_temp.startswith('\t}'):
                        break
                    else:
                        leader_class.append(line_temp.replace('\t', ''))
            if line_temp.startswith('}'):
                if name != None and random:
                    #print(tech_name, tier)
                    trigger = trigger.replace('from', 'owner')
                    for j in leader_class:
                        with open(os.path.join(work_dir, 'leader ' + j), 'a') as f2:
                            f2.write('5 = {{\n\tif = {{\n\t\tlimit = {{\n\t\t\tnot = {{\n\t\t\t\thas_trait = {0}\n\t\t\t}}{1}\n\t\t}}\n\t\tadd_trait = {0}\n\t\tset_timed_leader_flag = {{\n\t\t\tflag = gained_{0}\n\t\t\tdays = 7\n\t\t}}\n\t\tset_timed_leader_flag = {{\n\t\t\tflag = has_gained_trait\n\t\t\tdays = 7\n\t\t}}\n\t}}\n}}\n'.format(name, trigger))
                name = None
                random = True
                leader_class = []
                trigger = ''
        i += 1

make_trait(glob.glob(target_dir + '\\*.txt'))
print('done')